In [1]:
from IPython.display import display, Image
from prov.model import ProvDocument
from prov.dot import prov_to_dot
from shortid import ShortId

ModuleNotFoundError: No module named 'prov'

In [1]:
#Specific Weather station steps 
#instrument/measures/parameter
#logger/records/observation
#Iridium/sends/data
#Obsinn/converts to kvalobsformat/data
#Kvalobs/controls/data
#kdvh/stores/data

vocab = {
    'obo:SIO_001054': 'measuring',
    'obo:SIO_000396': 'file',
    'obo:NCIT_C15311': 'Quality Control',
    'obo:NCIT_C48177': 'Processing',
    'obo:OBI_0200170': 'averaging data transformation',
    #add air pressure at sea level 
    'sopropp:SeaLevelPressure': 'air pressure at sea level',
    'vnc:CFSN0022' : 'air pressure at sea level'
    #what is identifier SDN:P07::CFSN0022?
}

sid = ShortId()
prov = ProvDocument()

#ontology namespace 
prov.add_namespace('obo', 'http://purl.obolibrary.org/obo/')
prov.add_namespace('metno', 'http://metno.no/')
prov.add_namespace('pidinst', 'http://pidinst.org/')
#sweet ontology prop pressure
prov.add_namespace('sopropp','http://sweetontology.net/propPressure/')
prov.add_namespace('vcn','http://vocab.nerc.ac.uk/collection/P07/current/')
prov.add_namespace('rdfs', 'http://www.w3.org/2000/01/rdf-schema#')

#first set of triples: instruments measures parameter
#baromenter/measures/air_pressure_at_sea_level

#the barometer
ag1 = prov.agent('metno:instrument/{}'.format(sid.generate()), 
                 other_attributes={'prov:type': 'pidinst:MeasuringInstrument',
                                   'pidinst:InstrumentType': 'barometer'})

#measures
ac1 = prov.activity('metno:activities/{}'.format(sid.generate()), '1996-01-01T03:00:00', 
                    other_attributes={'prov:label': vocab['obo:SIO_001054'], 
                                      'prov:type': 'obo:SIO_001054'})

#                                                                                                          
et1 = prov.entity('metno:variables/{}'.format(sid.generate()), 
                  other_attributes={'prov:type': 'sopropp:SeaLevelPressure'})

#ac2 = prov.activity('nilu:activities/{}'.format(sid.generate()), '2019-12-03T12:10:09', other_attributes={'prov:label': vocab['obo:NCIT_C15311'], 
#                                                                                                                              'prov:type': 'obo:NCIT_C15311'})
#ac3 = prov.activity('nilu:activities/{}'.format(sid.generate()), '2019-12-03T12:10:09', other_attributes={'prov:label': vocab['obo:NCIT_C48177'], 
#                                                                                                                              'prov:type': 'obo:NCIT_C48177'})
#ac4 = prov.activity('nilu:activities/{}'.format(sid.generate()), '2019-12-03T12:20:12', other_attributes={'prov:label': vocab['obo:OBI_0200170'], 
#                                                                                                                             'prov:type': 'obo:OBI_0200170'})
    
prov.wasGeneratedBy(et1, ac1)
prov.wasAssociatedWith(ac1, ag1)

prov.serialize('data-provenance-sios.ttl', format='rdf', rdf_format='ttl')

dot = prov_to_dot(prov)
display(Image(dot.create_png()))

IndentationError: unexpected indent (<ipython-input-1-0fc774e25310>, line 51)